In [1]:
import os
Root = ("D:/Pinnacle/New_folder/speech-emotion-recognition")
os.chdir(Root)

In [2]:
ls

 Volume in drive D is New Volume
 Volume Serial Number is E235-0182

 Directory of D:\Pinnacle\New_folder\speech-emotion-recognition

24-05-2023  13:50    <DIR>          .
24-05-2023  13:50    <DIR>          ..
23-03-2018  02:24    <DIR>          Actor_01
23-03-2018  02:24    <DIR>          Actor_02
23-03-2018  02:24    <DIR>          Actor_03
23-03-2018  02:24    <DIR>          Actor_04
23-03-2018  02:25    <DIR>          Actor_05
23-03-2018  02:25    <DIR>          Actor_06
23-03-2018  02:25    <DIR>          Actor_07
23-03-2018  02:26    <DIR>          Actor_08
23-03-2018  02:26    <DIR>          Actor_09
23-03-2018  02:26    <DIR>          Actor_10
23-03-2018  02:26    <DIR>          Actor_11
23-03-2018  02:27    <DIR>          Actor_12
23-03-2018  02:27    <DIR>          Actor_13
23-03-2018  02:27    <DIR>          Actor_14
23-03-2018  02:28    <DIR>          Actor_15
23-03-2018  02:28    <DIR>          Actor_16
23-03-2018  02:28    <DIR>          Actor_17
23-03-2018  02:28    <DI

# Library Import

In [3]:
import librosa
import soundfile
import os,glob,pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [8]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [9]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [10]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("D:/Pinnacle/New_folder/speech-emotion-recognition/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [11]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [12]:
x_train

array([[-6.27527405e+02,  5.33894958e+01, -1.14486599e+01, ...,
         9.81572015e-08,  1.39831098e-08,  6.95362701e-09],
       [-5.27008179e+02,  5.93238449e+01, -3.09062500e+01, ...,
         1.25772306e-06,  3.85467985e-07,  2.83241004e-08],
       [-6.52832153e+02,  7.95012589e+01, -2.89284439e+01, ...,
         2.20646292e-08,  8.29448954e-09,  7.72867725e-09],
       ...,
       [-4.68942535e+02,  4.98182716e+01, -2.75305996e+01, ...,
         1.25355632e-06,  1.32783384e-07,  1.07083471e-08],
       [-5.30277283e+02,  9.47237320e+01, -1.58368969e+01, ...,
         4.47270565e-07,  1.22205975e-07,  1.75634849e-08],
       [-5.16981506e+02,  6.73251495e+01, -2.13805180e+01, ...,
         1.90207095e-06,  3.97173380e-07,  1.05029443e-07]])

In [13]:
print(x_train.shape[0],x_test.shape[0])

414 138


In [14]:
print(f'Features extracted: {x_train.shape[1]} ')

Features extracted: 180 


In [15]:
model = MLPClassifier(alpha=0.01,batch_size=256,epsilon=1e-08,hidden_layer_sizes=(300,),learning_rate='adaptive',max_iter=500)

In [16]:
# fitting model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [17]:
# predicting
y_pred = model.predict(x_test)
y_pred

array(['happy', 'happy', 'fearful', 'calm', 'fearful', 'fearful',
       'fearful', 'calm', 'happy', 'fearful', 'calm', 'fearful', 'happy',
       'calm', 'fearful', 'calm', 'fearful', 'calm', 'calm', 'fearful',
       'happy', 'calm', 'happy', 'fearful', 'fearful', 'calm', 'happy',
       'fearful', 'calm', 'fearful', 'calm', 'calm', 'happy', 'calm',
       'happy', 'calm', 'fearful', 'fearful', 'happy', 'calm', 'calm',
       'calm', 'fearful', 'fearful', 'calm', 'fearful', 'calm', 'calm',
       'fearful', 'calm', 'fearful', 'happy', 'happy', 'happy', 'happy',
       'happy', 'fearful', 'happy', 'happy', 'calm', 'calm', 'happy',
       'happy', 'happy', 'calm', 'happy', 'happy', 'fearful', 'happy',
       'happy', 'fearful', 'happy', 'fearful', 'fearful', 'happy', 'calm',
       'fearful', 'fearful', 'happy', 'calm', 'fearful', 'fearful',
       'happy', 'calm', 'happy', 'happy', 'calm', 'happy', 'calm',
       'happy', 'calm', 'calm', 'fearful', 'happy', 'fearful', 'fearful',
     

In [18]:
accuracy = accuracy_score(y_true=y_test,y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 92.75%


In [19]:
from sklearn.metrics import accuracy_score, f1_score

f1_score(y_test,y_pred,average=None)

array([0.88888889, 0.97959184, 0.90721649])

In [20]:
# saving the model
import pickle
with open('modelForPrediction.sav','wb') as f:
    pickle.dump(model,f)

In [22]:
filename = 'modelforprediction.sav'

loaded_model = pickle.load(open(filename,'rb'))

feature = extract_feature("D:/Pinnacle/New_folder/speech-emotion-recognition/Actor_17/03-02-01-01-01-02-17.wav",mfcc=True, chroma=True, mel=True)

feature = feature.reshape(1,-1)

prediction = loaded_model.predict(feature)

prediction

array(['fearful'], dtype='<U7')